In [3]:
# import dependencies
import tensorflow as tf
import numpy as np
import cv2

import dlib
import pickle

ModuleNotFoundError: No module named 'cv2'

In [2]:
# create model
def get_model():
    backbone = tf.keras.applications.EfficientNetB2(
        input_shape=(96, 96, 3),
        include_top=False,
        weights=None
    )

    model = tf.keras.Sequential([
        backbone,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(7, activation='softmax')
    ])
    
    return model

In [3]:
# load the saved weights 
model = get_model()
model.load_weights("best_weights.h5") 

In [4]:
# load LabelEncoder 
def load_object(name):
    pickle_obj = open(f"{name}.pck","rb")
    obj = pickle.load(pickle_obj)
    return obj

Le = load_object("LabelEncoder")

In [5]:
# create some functions for our use
def ProcessImage(image):
    image = tf.convert_to_tensor(image)
    image = tf.image.resize(image , [96 , 96] , method="bilinear")
    image = tf.expand_dims(image , 0)
    return image

def RealtimePrediction(image , model, encoder_):
    prediction = model.predict(image)
    prediction = np.argmax(prediction , axis = 1)
    return encoder_.inverse_transform(prediction)[0]

# bounding box
def rect_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)

In [1]:
VideoCapture = cv2.VideoCapture(0) # connect to frame with real time "0" is conncet to local cam

detector = dlib.get_frontal_face_detector() # detect face in both RGB and Greyscale

while True :
    
    ret , frame = VideoCapture.read()  # returns boolean and numpy array
    
    if not ret :
        break

    gray = cv2.cvtColor( frame , cv2.COLOR_BGR2GRAY) # convert to greyscale

    rects = detector(gray , 0) # pass image to detector

    if len(rects) >= 1 : # check if any face is inside camera frame
        for rect in rects : # if any face, loop inside each rectangle
            (x , y , w , h) = rect_to_bb(rect) # get co-ordinates of face
            img = gray[y-10 : y+h+10 , x-10 : x+w+10] # crop face out of the image using co-ordinates
            
            if img.shape[0] == 0 or img.shape[1] == 0 : # check if frame is not empty
                cv2.imshow("Frame", frame) 
                
            else : 
                img = cv2.cvtColor(img , cv2.COLOR_GRAY2RGB) # while training we used 3 channels images
                img = ProcessImage(img) # return tensor with 4 dimensions
                out = RealtimePrediction(img , model , Le) # pass the tensor to a function along with model and label encoder object
                cv2.rectangle(frame, (x, y), (x+w, y+h),(0, 255, 0), 2) # does prediction and return in a string format, we used it to draw rectangle around the face
                z = y - 15 if y - 15 > 15 else y + 15
                cv2.putText(frame, str(out), (x, z), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2) # add text at top of rectangle
                
        cv2.imshow("Frame", frame)
            
    else :
        cv2.imshow("Frame", frame)
        
    if cv2.waitKey(1) & 0xFF == ord('q'): # to stop the predictions
        break

# stop the connections and distroy any windows created by cv2 on our machine
VideoCapture.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined